# Génération du Labyrinthe

> L'objectif de ce notebook est d'entrainer un modèle à générer des labyrinthes complexes et infinis en utilisant les cartes d'exemple disponibles dans ce même répertoire.

## Génération du dataset

Dans un premier temps, nous cherchons à obtenir un dataset de la forme suivante :
$$
\text{Voisinage} \rightarrow \text{Case}
$$
Pour ce faire, nous allons reprendre l'idée des filtres de convolution et balayer le labyrinthe avec un filtre de taille $n\times n$.

In [45]:
# -------------------------- CHARGEMENT DES DONNEES -------------------------- #
import pandas as pd
import os
from IPython.display import display, Markdown

data = {}
for root, dirs, files in os.walk("."):
    for filename in files:
        if filename.endswith(".csv"):
            last_character = filename[-5]
            simple_filename = filename.split(".")[0]
            if simple_filename not in data:
                data[simple_filename] = {}
            if last_character not in data[simple_filename]:
                csv = pd.read_csv(filename).to_numpy()
                if csv.shape[0] * csv.shape[1] != 0:
                    data[simple_filename][last_character] = csv

for data_name in data:
    display(Markdown(f"-> **{data_name}** : *{len(data[data_name])} layers*"))

-> **training_map** : *1 layers*

In [49]:
# --------------------------- CREATON DES DATASETS --------------------------- #
def create_dataset(data, window_size=2):
    if data.shape[0] < window_size or data.shape[1] < window_size:
        raise ValueError("Window size is too big for this data")
    dataset = pd.DataFrame(
        [
            data[i : i + window_size, j : j + window_size].flatten()
            for i in range(0, data.shape[0] - window_size + 1)
            for j in range(0, data.shape[1] - window_size + 1)
        ]
    )
    dataset["label"] = dataset[dataset.columns[(window_size**2 - 1) // 2]]
    dataset.drop(dataset.columns[(window_size**2 - 1) // 2], axis=1, inplace=True)
    return dataset


datasets = {
    data_name: {
        last_character: create_dataset(data[data_name][last_character])
        for last_character in data[data_name]
    }
    for data_name in data
}
for dataset in datasets:
    for layer in datasets[dataset]:
        display(
            Markdown(
                f"-> **{dataset}** : *{layer} : {len(datasets[dataset][layer])} samples*"
            )
        )

-> **training_map** : *1 : 9 samples*